In [ ]:
!pip3 install fuzzywuzzy
!pip3 install transformers==2.11.0

In [ ]:
import pandas as pd
import numpy as np
import seaborn as snb
import warnings
import matplotlib.pyplot as plt
import re
import os
import string
from gensim.models import KeyedVectors, Word2Vec
from collections import Counter
from textblob import TextBlob
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from fuzzywuzzy import fuzz
from collections import Counter
import tensorflow as tf
from sklearn.model_selection import GroupKFold
import tensorflow_hub as hub
import random as rn
from scipy.stats import spearmanr
from transformers import XLNetConfig, TFXLNetModel, XLNetTokenizer, TFXLNetMainLayer
SEED = 21
os.environ['PYTHONHASHSEED']=str(SEED)
os.environ['TF_CUDNN_DETERMINISTIC'] = '1'
warnings.filterwarnings('ignore')

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
if os.getcwd() != '/content/drive/My Drive/google_qa':
    os.chdir('./drive/My Drive/google_qa')

In [ ]:
class XLNet(TFXLNetModel):
    def __init__(self, config, *inputs, **kwrgs):
        super(XLNet, self).__init__(config, *inputs, **kwrgs)
        self.transformer = TFXLNetMainLayer(config, name = 'transformer')
        for i in range(1, 3):
            self.transformer.layer[-i].trainable = False
    def call(self, inputs, **kwrgs):
        outputs = self.transformer(inputs)
        hidden_states = outputs[1]
        h12 = hidden_states[-1][:, 0, :]
        h11 = hidden_states[-2][:, 0, :]
        h10 = hidden_states[-3][:, 0, :]
        h9 = hidden_states[-4][:, 0, :]
        concat = tf.keras.layers.Concatenate(axis = -1)([h9, h10, h11, h12])
        return concat
def transformer_model():
    id_1 = tf.keras.Input(shape = (512), dtype = tf.int32)
    id_2 = tf.keras.Input(shape = (512), dtype = tf.int32)

    type_id_1 = tf.keras.Input(shape = (512), dtype = tf.int32)
    type_id_2 = tf.keras.Input(shape = (512), dtype = tf.int32)
        
    a1 = tf.keras.Input(shape = (512), dtype = tf.int32)
    a2 = tf.keras.Input(shape = (512), dtype = tf.int32)

    config = XLNetConfig.from_pretrained('xlnet/config-xlnet-base-cased', output_hidden_states = True)
    transformer = XLNet.from_pretrained('xlnet/model-xlnet-base-cased', config = config)
                                                

    out_1 = transformer({'input_ids':id_1, 'attention_mask':a1, 'token_type_ids':type_id_1})
    out_2 = transformer({'input_ids':id_2, 'attention_mask':a2, 'token_type_ids':type_id_2})
    
    concat = tf.keras.layers.Concatenate(axis = -1)([out_1, out_2])
    drop = tf.keras.layers.Dropout(rate = 0.1)(concat)
    dense = tf.keras.layers.Dense(30, activation = 'sigmoid')(drop)
    
    return tf.keras.Model(inputs = [id_1, id_2, type_id_1, type_id_2, a1, a2], outputs = [dense])

In [ ]:
xlnet_tokenizer = XLNetTokenizer.from_pretrained('xlnet/tokenizer')

In [ ]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

In [ ]:
def tokenize_input(tokenizer, s1, s2, tags, data_name, max_length, tokenizer_name = 'bert'):
  if s2 is not None:
    x = tokenizer.encode_plus(s1, s2, pad_to_max_length=False)
    if len(x['input_ids']) > max_length:
      segment_1 = int(0.25*max_length)
      x['input_ids'] = x['input_ids'][:segment_1] + x['input_ids'][-(max_length-segment_1):]
      x['attention_mask'] = x['attention_mask'][:segment_1] + x['attention_mask'][-(max_length-segment_1):]
      x['token_type_ids'] = x['token_type_ids'][:segment_1] + x['token_type_ids'][-(max_length-segment_1):]
    else:
      diff = max_length - len(x['input_ids'])
      if tokenizer_name == 'xlnet':
        x['input_ids'] = [tokenizer.pad_token_id]*diff + x['input_ids']
        x['attention_mask'] = [0]*diff + x['attention_mask']
        x['token_type_ids'] = [tokenizer.pad_token_type_id]*diff + x['token_type_ids']
      else:
        x['input_ids'] = x['input_ids'] + [tokenizer.pad_token_id]*diff
        x['attention_mask'] = x['attention_mask'] + [0]*diff
        x['token_type_ids'] = x['token_type_ids'] + [0]*diff
      
  else:
    x = tokenizer.encode_plus(s1)
    if len(x['input_ids']) > max_length:
      segment_1 = int(0.25*max_length)
      x['input_ids'] = x['input_ids'][:segment_1] + x['input_ids'][-(max_length-segment_1):]
      x['attention_mask'] = x['attention_mask'][:segment_1] + x['attention_mask'][-(max_length-segment_1):]
      x['token_type_ids'] = x['token_type_ids'][:segment_1] + x['token_type_ids'][-(max_length-segment_1):]
    else:
      diff = max_length - len(x['input_ids'])
      if tokenizer_name == 'xlnet':
        x['input_ids'] = [tokenizer.pad_token_id]*diff + x['input_ids']
        x['attention_mask'] = [0]*diff + x['attention_mask']
        x['token_type_ids'] = [tokenizer.pad_token_type_id]*diff + x['token_type_ids']
      else:
        x['input_ids'] = x['input_ids'] + [tokenizer.pad_token_id]*diff
        x['attention_mask'] = x['attention_mask'] + [0]*diff
        x['token_type_ids'] = x['token_type_ids'] + [tokenizer.pad_token_type_id]*diff
  
  data[data_name][tags[0]].append(x['input_ids']) 
  data[data_name][tags[1]].append(x['token_type_ids'])
  data[data_name][tags[2]].append(x['attention_mask']) 

data = {}
# ******************************************XLNET*************************************************************************************
data['xlnet_train_t_a'] = {}
data['xlnet_train_q_a'] = {}
data['xlnet_train_t_q'] = {}
data['xlnet_train_q'] = {}
data['xlnet_train_a'] = {}

data['xlnet_test_t_a'] = {}
data['xlnet_test_q_a'] = {}
data['xlnet_test_t_q'] = {}
data['xlnet_test_q'] = {}
data['xlnet_test_a'] = {}

tags = ['input_ids', 'token_type_ids', 'attention_masks']
data['xlnet_train_t_a'][tags[0]], data['xlnet_train_t_a'][tags[1]], data['xlnet_train_t_a'][tags[2]] = [], [], []
data['xlnet_train_q_a'][tags[0]], data['xlnet_train_q_a'][tags[1]], data['xlnet_train_q_a'][tags[2]] = [], [], []
data['xlnet_train_t_q'][tags[0]], data['xlnet_train_t_q'][tags[1]], data['xlnet_train_t_q'][tags[2]] = [], [], []
data['xlnet_train_q'][tags[0]], data['xlnet_train_q'][tags[1]], data['xlnet_train_q'][tags[2]] = [], [], []
data['xlnet_train_a'][tags[0]], data['xlnet_train_a'][tags[1]], data['xlnet_train_a'][tags[2]] = [], [], []

data['xlnet_test_t_a'][tags[0]], data['xlnet_test_t_a'][tags[1]], data['xlnet_test_t_a'][tags[2]] = [], [], []
data['xlnet_test_q_a'][tags[0]], data['xlnet_test_q_a'][tags[1]], data['xlnet_test_q_a'][tags[2]] = [], [], []
data['xlnet_test_t_q'][tags[0]], data['xlnet_test_t_q'][tags[1]], data['xlnet_test_t_q'][tags[2]] = [], [], []
data['xlnet_test_q'][tags[0]], data['xlnet_test_q'][tags[1]], data['xlnet_test_q'][tags[2]] = [], [], []
data['xlnet_test_a'][tags[0]], data['xlnet_test_a'][tags[1]], data['xlnet_test_a'][tags[2]] = [], [], []


for i in range(train_df.shape[0]):
  tokenize_input(xlnet_tokenizer, train_df.loc[i, 'question_title'], train_df.loc[i, 'answer'], tags, 'xlnet_train_t_a', 512, 'xlnet')
  tokenize_input(xlnet_tokenizer, train_df.loc[i, 'question_body'], train_df.loc[i, 'answer'], tags, 'xlnet_train_q_a', 512, 'xlnet')
  tokenize_input(xlnet_tokenizer, train_df.loc[i, 'question_title'], train_df.loc[i, 'question_body'], tags, 'xlnet_train_t_q', 512, 'xlnet')
  tokenize_input(xlnet_tokenizer, train_df.loc[i, 'question_body'], None, tags, 'xlnet_train_q', 512, 'xlnet')
  tokenize_input(xlnet_tokenizer, train_df.loc[i, 'answer'], None, tags, 'xlnet_train_a', 512, 'xlnet')
for i in range(test_df.shape[0]):
  tokenize_input(xlnet_tokenizer, test_df.loc[i, 'question_title'], test_df.loc[i, 'answer'], tags, 'xlnet_test_t_a', 512, 'xlnet')
  tokenize_input(xlnet_tokenizer, test_df.loc[i, 'question_body'], test_df.loc[i, 'answer'], tags, 'xlnet_test_q_a', 512, 'xlnet')
  tokenize_input(xlnet_tokenizer, test_df.loc[i, 'question_title'], test_df.loc[i, 'question_body'], tags, 'xlnet_test_t_q', 512, 'xlnet')
  tokenize_input(xlnet_tokenizer, test_df.loc[i, 'question_body'], None, tags, 'xlnet_test_q', 512, 'xlnet')
  tokenize_input(xlnet_tokenizer, test_df.loc[i, 'answer'], None, tags, 'xlnet_test_a', 512, 'xlnet')

for key, _ in data.items():
  for k, _ in data[key].items():
    data[key][k] = np.array(data[key][k])

In [ ]:
'''Some contractions, rules and token replace dictionaries which will help in text preprocessing'''
contractions = {
"ain't": "is not",
"aren't": "are not",
"can't": "cannot",
"can't've": "cannot have",
"'cause": "because",
"could've": "could have",
"couldn't": "could not",
"couldn't've": "could not have",
"didn't": "did not",
"doesn't": "does not",
"don't": "do not",
"hadn't": "had not",
"hadn't've": "had not have",
"hasn't": "has not",
"haven't": "have not",
"he'd": "he would",
"he'd've": "he would have",
"he'll": "he will",
"he'll've": "he he will have",
"he's": "he is",
"how'd": "how did",
"how'd'y": "how do you",
"how'll": "how will",
"how's": "how is",
"I'd": "I would",
"i'd've": "I would have",
"i'll": "I will",
"i'll've": "I will have",
"i'm": "I am",
"i've": "I have",
"i'd": "i would",
"i'd've": "i would have",
"i'll": "i will",
"i'll've": "i will have",
"i'm": "i am",
"i've": "i have",
"isn't": "is not",
"its":"it is",
"it'd": "it would",
"it'd've": "it would have",
"it'll": "it will",
"it'll've": "it will have",
"it's": "it is",
"let's": "let us",
"ma'am": "madam",
"mayn't": "may not",
"might've": "might have",
"mightn't": "might not",
"mightn't've": "might not have",
"must've": "must have",
"mustn't": "must not",
"mustn't've": "must not have",
"needn't": "need not",
"needn't've": "need not have",
"o'clock": "of the clock",
"oughtn't": "ought not",
"oughtn't've": "ought not have",
"shan't": "shall not",
"sha'n't": "shall not",
"shan't've": "shall not have",
"she'd": "she would",
"she'd've": "she would have",
"she'll": "she will",
"she'll've": "she will have",
"she's": "she is",
"should've": "should have",
"shouldn't": "should not",
"shouldn't've": "should not have",
"so've": "so have",
"so's": "so as",
"that'd": "that would",
"that'd've": "that would have",
"that's": "that is",
"there'd": "there would",
"there'd've": "there would have",
"there's": "there is",
"they'd": "they would",
"they'd've": "they would have",
"they'll": "they will",
"they'll've": "they will have",
"they're": "they are",
"they've": "they have",
"to've": "to have",
"wasn't": "was not",
"we'd": "we would",
"we'd've": "we would have",
"we'll": "we will",
"we'll've": "we will have",
"we're": "we are",
"we've": "we have",
"weren't": "were not",
"what'll": "what will",
"what'll've": "what will have",
"what're": "what are",
"what's": "what is",
"what've": "what have",
"when's": "when is",
"when've": "when have",
"where'd": "where did",
"where's": "where is",
"where've": "where have",
"who'll": "who will",
"who'll've": "who will have",
"who's": "who is",
"who've": "who have",
"why's": "why is",
"why've": "why have",
"will've": "will have",
"won't": "will not",
"won't've": "will not have",
"would've": "would have",
"wouldn't": "would not",
"wouldn't've": "would not have",
"y'all": "you all",
"y'all'd": "you all would",
"y'all'd've": "you all would have",
"y'all're": "you all are",
"y'all've": "you all have",
"you'd": "you would",
"you'd've": "you would have",
"you'll": "you will",
"you'll've": "you will have",
"you're": "you are",
"you've": "you have"
}

rules = {
    "'t": " not",
    "'cause": " because",
    "'ve": " have",
    "'t": " not",
    "'s": " is",
    "'d": " had"
}

punctuations = [',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", 
                '$', '&', '/', '[', ']','>', '%', '=', '#', '*', '+', '\\', '•', '~', 
                '@', '£', '·', '_', '{', '}', '©', '^','®', '`', '<', '→', '°', '€', '™', 
                '›', '♥', '←', '×', '§', '″', '′', 'Â', '█','½', 'à', '…', '“', '★', '”', 
                '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶','↑', '±', '¿', '▾', '═', 
                '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼','⊕', '▼', '▪', '†',
                '■', '’', '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲','è', '¸', '¾', 
                'Ã', '⋅', '‘', '∞', '∙', '）', '↓', '、', '│', '（', '»', '，', '♪','╩', 
                '╚', '³', '・', '╦', '╣', '╔', '╗', '▬', '❤', 
                'ï', 'Ø', '¹', '≤', '‡', '√'] + list(string.punctuation)
token_replace = {
    'usepackage':'latex',
    'orf19':'gene',
    'documentclass':'latex',
    'magento':'open-source e-commerce',
    'appium':'web-app',
    'tikz':'programming language',
    'tikzpicture':'programming language',
    'openvpn':'vpn',
    'httpclient':'http client',
    'arraylist':'array list',
    'jsonobject': 'json',
    'artifactid':'xml',
    'hwnd':'os'
}
punctuations_in_embeddings = {',', '.', '"', ':', ')', '(', '-', '!', '?', '|', ';', "'", '$', '&', '/', '[', ']', 
                '>', '%', '=', '#', '*', '+', '\\', '•', '~', '@', '£', '·', '{', '}', '©', '^', '®', 
                '<', '→', '°', '€', '™', '›', '♥', '←', '×', '§', '″', '′', 'Â', '█', '½', 'à', '…', 
                '“', '★', '”', '–', '●', 'â', '►', '−', '¢', '²', '¬', '░', '¶', '↑', '±', '¿', '═', 
                '¦', '║', '―', '¥', '▓', '—', '‹', '─', '▒', '：', '¼', '⊕', '▼', '▪', '†', '■', '’', 
                '▀', '¨', '▄', '♫', '☆', 'é', '¯', '♦', '¤', '▲', 'è', '¸', '¾', 'Ã', '⋅', '‘', '∞', 
                '∙', '）', '↓', '、', '│', '（', '»', '，', '♪', '╩', '╚', '³', '・', '╦', '╣', '╔',
                '╗', '▬', '❤', 'ï', 'Ø', '¹', '≤', '‡', '√', '!', '"', '#', '$', '%', '&', "'", '(', ')', 
                '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '{', 
                '|', '}', '~'}

In [ ]:
'''Pipeline for text preprocessing. Check coverage computes what %age of tokens in the text data 
are covered by Embeddings.
preprocess_text is used to clean text.
'''
def improve_text(dframes):
    def preprocess_text(s):
        s = s.lower()
        '''Expanding contractions...'''
        for key, value in contractions.items():
            s = s.replace(key, f' {value} ')
        for key, value in rules.items():
            s = s.replace(key, f' {value} ')
        '''Fixing punctuations...'''
        for punct in punctuations:
            if punct in punctuations_in_embeddings:
                s = s.replace(punct, f' {punct} ')
            else:
                s = s.replace(punct, ' ')
        '''Replacing few tokens with its similar word/group of words...'''
        for key, value in token_replace.items():
            s = s.replace(key, value)
        '''Removing HTML tags'''
        s = re.sub('<.*?>', ' ', s)
        s = re.sub('\s+', ' ', s)
        return s
    print("Applying Preprocessing.....")
    for each_df in dframes:
        each_df['clean_title'] = each_df['question_title'].apply(preprocess_text)
        each_df['clean_question_body'] = each_df['question_body'].apply(preprocess_text)
        each_df['clean_answer'] = each_df['answer'].apply(preprocess_text)
    print("Preprocessing Finished")

In [ ]:
improve_text([train_df, test_df])

In [ ]:
def fetch_words(s, use_stopwords = False, ignore_stopwords = False):
    if ignore_stopwords == True:
        return {word for word in s.split() if word not in stop_words}
    elif use_stopwords:
        return {word for word in s.split() if word in stop_words}
    else:
        return set(s.split())
def text_features(row, use_stopwords, is_token, return_max):
    t1, t2 = None, None
    if use_stopwords == True:
        t1 = fetch_words(row[0], use_stopwords = True)
        t2 = fetch_words(row[1], use_stopwords = True)
    elif is_token == True:
        t1 = fetch_words(row[0], ignore_stopwords = True)
        t2 = fetch_words(row[1], ignore_stopwords = True)
    else:
        t1 = fetch_words(row[0])
        t2 = fetch_words(row[1])
    if return_max:
        try:
            ans = len(t1.intersection(t2))/max(len(t1), len(t2))
        except:
            return 0
    else:
        try:
            ans = len(t1.intersection(t2))/min(len(t1), len(t2))
        except:
            return 0
    return ans

def get_ratio(row, is_token = False):
    if is_token == True:
        t1 = len(set([word for word in row[0].split() if word not in stop_words]))
        t2 = len(set([word for word in row[1].split() if word not in stop_words]))
    t1 = len(set([word for word in row[0].split()]))
    t2 = len(set([word for word in row[1].split()]))
    try:
        return t1/t2
    except:
        return 0
def get_fuzzy_ratio(row, ratio, partial_ratio, token_sort_ratio, token_set_ratio):
    if ratio:
        return fuzz.ratio(row[0], row[1])/100
    elif partial_ratio:
        return fuzz.partial_ratio(row[0], row[1])/100
    elif token_sort_ratio:
        return fuzz.token_sort_ratio(row[0], row[1])/100
    else:
        return fuzz.token_set_ratio(row[0], row[1])/100

In [ ]:
feature_engg_cols = ['cwc_min_title_question_body',
       'cwc_max_title_question_body', 'csc_min_title_question_body',
       'csc_max_title_question_body', 'ctc_min_title_question_body',
       'ctc_max_title_question_body', 'cwc_min_title_answer',
       'cwc_max_title_answer', 'csc_min_title_answer', 'csc_max_title_answer',
       'ctc_min_title_answer', 'ctc_max_title_answer',
       'cwc_min_question_body_answer', 'cwc_max_question_body_answer',
       'csc_min_question_body_answer', 'csc_max_question_body_answer',
       'ctc_min_question_body_answer', 'ctc_max_question_body_answer',
       'word_title_question_body_ratio', 'token_title_question_body_ratio',
       'word_question_body_answer_ratio', 'token_question_body_answer_ratio',
       'word_title_answer_ratio', 'token_title_answer_ratio',
       'ratio_title_question_body', 'partial_ratio_title_question_body',
       'token_sort_ratio_title_question_body',
       'token_set_ratio_title_question_body', 'ratio_question_body_answer',
       'partial_ratio_question_body_answer',
       'token_sort_ratio_question_body_answer',
       'token_set_ratio_question_body_answer', 'ratio_title_answer',
       'partial_ratio_title_answer', 'token_sort_ratio_title_answer',
       'token_set_ratio_title_answer', 'question_title_polarity', 'question_body_polarity', 'answer_polarity']

In [ ]:
def get_polarity(s):
    return TextBlob(s).sentiment.polarity
train_df['question_title_polarity'] = train_df.clean_title.apply(get_polarity)
train_df['question_body_polarity'] = train_df.clean_question_body.apply(get_polarity)
train_df['answer_polarity'] = train_df.clean_answer.apply(get_polarity)
test_df['question_title_polarity'] = test_df.clean_title.apply(get_polarity)
test_df['question_body_polarity'] = test_df.clean_question_body.apply(get_polarity)
test_df['answer_polarity'] = test_df.clean_answer.apply(get_polarity)
##################################################################################################################
train_df['question_title_count'] = train_df.clean_title.apply(lambda x: len(x.split()))
train_df['question_body_count'] = train_df.clean_question_body.apply(lambda x: len(x.split()))
train_df['answer_count'] = train_df.clean_answer.apply(lambda x: len(x.split()))
test_df['question_title_count'] = test_df.clean_title.apply(lambda x: len(x.split()))
test_df['question_body_count'] = test_df.clean_question_body.apply(lambda x: len(x.split()))
test_df['answer_count'] = test_df.clean_answer.apply(lambda x: len(x.split()))

In [ ]:
##################################        TRAIN        ############################################################
train_df['cwc_min_title_question_body'] = train_df[['clean_title', 'clean_question_body']].apply(func = text_features, 
                                                axis = 1, args = (False, False, False))
train_df['cwc_max_title_question_body'] = train_df[['clean_title', 'clean_question_body']].apply(func = text_features, 
                                                axis = 1, args = (False, False, True))
train_df['csc_min_title_question_body'] = train_df[['clean_title', 'clean_question_body']].apply(func = text_features, 
                                                axis = 1, args = (True, False, False))
train_df['csc_max_title_question_body'] = train_df[['clean_title', 'clean_question_body']].apply(func = text_features, 
                                                axis = 1, args = (True, False, True))
train_df['ctc_min_title_question_body'] = train_df[['clean_title', 'clean_question_body']].apply(func = text_features, 
                                                axis = 1, args = (False, True, False))
train_df['ctc_max_title_question_body'] = train_df[['clean_title', 'clean_question_body']].apply(func = text_features, 
                                                axis = 1, args = (False, True, True))
train_df['cwc_min_title_answer'] = train_df[['clean_title', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, False, False))
train_df['cwc_max_title_answer'] = train_df[['clean_title', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, False, True))
train_df['csc_min_title_answer'] = train_df[['clean_title', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (True, False, False))
train_df['csc_max_title_answer'] = train_df[['clean_title', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (True, False, True))
train_df['ctc_min_title_answer'] = train_df[['clean_title', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, True, False))
train_df['ctc_max_title_answer'] = train_df[['clean_title', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, True, True))
train_df['cwc_min_question_body_answer'] = train_df[['clean_question_body', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, False, False))
train_df['cwc_max_question_body_answer'] = train_df[['clean_question_body', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, False, True))
train_df['csc_min_question_body_answer'] = train_df[['clean_question_body', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (True, False, False))
train_df['csc_max_question_body_answer'] = train_df[['clean_question_body', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (True, False, True))
train_df['ctc_min_question_body_answer'] = train_df[['clean_question_body', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, True, False))
train_df['ctc_max_question_body_answer'] = train_df[['clean_question_body', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, True, True))

train_df['word_title_question_body_ratio'] = train_df[['clean_title', 'clean_question_body']].apply(
    func = get_ratio, axis = 1, args = (False, ))
train_df['token_title_question_body_ratio'] = train_df[['clean_title', 'clean_question_body']].apply(
    func = get_ratio, axis = 1, args = (True, ))
train_df['word_question_body_answer_ratio'] = train_df[['clean_question_body', 'clean_answer']].apply(
    func = get_ratio, axis = 1, args = (False, ))
train_df['token_question_body_answer_ratio'] = train_df[['clean_question_body', 'clean_answer']].apply(
    func = get_ratio, axis = 1, args = (True, ))
train_df['word_title_answer_ratio'] = train_df[['clean_title', 'clean_answer']].apply(func = get_ratio, 
                                                axis = 1, args = (False, ))
train_df['token_title_answer_ratio'] = train_df[['clean_title', 'clean_answer']].apply(func = get_ratio, 
                                                axis = 1, args = (True, ))

train_df['ratio_title_question_body'] = \
train_df[['clean_title', 'clean_question_body']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (True, False, False, False))
train_df['partial_ratio_title_question_body'] = \
train_df[['clean_title', 'clean_question_body']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, True, False, False))
train_df['token_sort_ratio_title_question_body'] = \
train_df[['clean_title', 'clean_question_body']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, False, True, False))
train_df['token_set_ratio_title_question_body'] = \
train_df[['clean_title', 'clean_question_body']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, False, False, True))
train_df['ratio_question_body_answer'] = \
train_df[['clean_question_body', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (True, False, False, False ))
train_df['partial_ratio_question_body_answer'] = \
train_df[['clean_question_body', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, True, False, False))
train_df['token_sort_ratio_question_body_answer'] = \
train_df[['clean_question_body', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, False, True, False))
train_df['token_set_ratio_question_body_answer'] = \
train_df[['clean_question_body', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, False, False, True))
train_df['ratio_title_answer'] = \
train_df[['clean_title', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (True, False, False, False))
train_df['partial_ratio_title_answer'] = \
train_df[['clean_title', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, True, False, False))
train_df['token_sort_ratio_title_answer'] = \
train_df[['clean_title', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, False, True, False))
train_df['token_set_ratio_title_answer'] = \
train_df[['clean_title', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, False, False, True))

##################################        TEST        ############################################################

test_df['cwc_min_title_question_body'] = test_df[['clean_title', 'clean_question_body']].apply(func = text_features, 
                                                axis = 1, args = (False, False, False))
test_df['cwc_max_title_question_body'] = test_df[['clean_title', 'clean_question_body']].apply(func = text_features, 
                                                axis = 1, args = (False, False, True))
test_df['csc_min_title_question_body'] = test_df[['clean_title', 'clean_question_body']].apply(func = text_features, 
                                                axis = 1, args = (True, False, False))
test_df['csc_max_title_question_body'] = test_df[['clean_title', 'clean_question_body']].apply(func = text_features, 
                                                axis = 1, args = (True, False, True))
test_df['ctc_min_title_question_body'] = test_df[['clean_title', 'clean_question_body']].apply(func = text_features, 
                                                axis = 1, args = (False, True, False))
test_df['ctc_max_title_question_body'] = test_df[['clean_title', 'clean_question_body']].apply(func = text_features, 
                                                axis = 1, args = (False, True, True))
test_df['cwc_min_title_answer'] = test_df[['clean_title', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, False, False))
test_df['cwc_max_title_answer'] = test_df[['clean_title', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, False, True))
test_df['csc_min_title_answer'] = test_df[['clean_title', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (True, False, False))
test_df['csc_max_title_answer'] = test_df[['clean_title', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (True, False, True))
test_df['ctc_min_title_answer'] = test_df[['clean_title', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, True, False))
test_df['ctc_max_title_answer'] = test_df[['clean_title', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, True, True))
test_df['cwc_min_question_body_answer'] = test_df[['clean_question_body', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, False, False))
test_df['cwc_max_question_body_answer'] = test_df[['clean_question_body', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, False, True))
test_df['csc_min_question_body_answer'] = test_df[['clean_question_body', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (True, False, False))
test_df['csc_max_question_body_answer'] = test_df[['clean_question_body', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (True, False, True))
test_df['ctc_min_question_body_answer'] = test_df[['clean_question_body', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, True, False))
test_df['ctc_max_question_body_answer'] = test_df[['clean_question_body', 'clean_answer']].apply(func = text_features, 
                                                axis = 1, args = (False, True, True))

test_df['word_title_question_body_ratio'] = test_df[['clean_title', 'clean_question_body']].apply(
    func = get_ratio, axis = 1, args = (False, ))
test_df['token_title_question_body_ratio'] = test_df[['clean_title', 'clean_question_body']].apply(
    func = get_ratio, axis = 1, args = (True, ))
test_df['word_question_body_answer_ratio'] = test_df[['clean_question_body', 'clean_answer']].apply(
    func = get_ratio, axis = 1, args = (False, ))
test_df['token_question_body_answer_ratio'] = test_df[['clean_question_body', 'clean_answer']].apply(
    func = get_ratio, axis = 1, args = (True, ))
test_df['word_title_answer_ratio'] = test_df[['clean_title', 'clean_answer']].apply(func = get_ratio, 
                                                axis = 1, args = (False, ))
test_df['token_title_answer_ratio'] = test_df[['clean_title', 'clean_answer']].apply(func = get_ratio, 
                                                axis = 1, args = (True, ))

test_df['ratio_title_question_body'] = \
test_df[['clean_title', 'clean_question_body']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (True, False, False, False))
test_df['partial_ratio_title_question_body'] = \
test_df[['clean_title', 'clean_question_body']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, True, False, False))
test_df['token_sort_ratio_title_question_body'] = \
test_df[['clean_title', 'clean_question_body']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, False, True, False))
test_df['token_set_ratio_title_question_body'] = \
test_df[['clean_title', 'clean_question_body']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, False, False, True))
test_df['ratio_question_body_answer'] = \
test_df[['clean_question_body', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (True, False, False, False ))
test_df['partial_ratio_question_body_answer'] = \
test_df[['clean_question_body', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, True, False, False))
test_df['token_sort_ratio_question_body_answer'] = \
test_df[['clean_question_body', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, False, True, False))
test_df['token_set_ratio_question_body_answer'] = \
test_df[['clean_question_body', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, False, False, True))
test_df['ratio_title_answer'] = \
test_df[['clean_title', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (True, False, False, False))
test_df['partial_ratio_title_answer'] = \
test_df[['clean_title', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, True, False, False))
test_df['token_sort_ratio_title_answer'] = \
test_df[['clean_title', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, False, True, False))
test_df['token_set_ratio_title_answer'] = \
test_df[['clean_title', 'clean_answer']].apply(func = get_fuzzy_ratio, 
                                                 axis = 1, args = (False, False, False, True))

In [23]:
def build_embedding_matrix(vocab, texts, embedd_size, model):
  n = len(vocab['token2id'])+1
  embedding_matrix = np.zeros((n, embedd_size))
  for text in texts:
    for key in text:
      word = key
      try:
        embedding_matrix[vocab['token2id'][word]] = model.wv[word]
        continue
      except:
        pass
  return embedding_matrix

def get_token_ids(texts, max_length):
  tokens = []
  for text in texts:
    tmp_tokens = []
    if len(text.split()) > max_length:
      for each in (text.split()[:(max_length//2)] + text.split()[-(max_length//2):]):
        tmp_tokens.append(vocab['token2id'].get(each, 0))
      tokens.append(tmp_tokens)
    else:
      for each in (text.split()[:max_length]):
        tmp_tokens.append(vocab['token2id'].get(each, 0))
      tokens.append(tmp_tokens)
  return tf.keras.preprocessing.sequence.pad_sequences(tokens, padding="post", maxlen=max_length)

train_df['clean_t_q'] = train_df['clean_title'] + ' ' + train_df['clean_question_body']
test_df['clean_t_q'] = test_df['clean_title'] + ' ' + test_df['clean_question_body']

In [ ]:
if os.path.exists('embeddings.npz') == False:
    train_df['combine_text'] = train_df.clean_title + ' ' + train_df.clean_question_body + ' ' + train_df.clean_answer
    test_df['combine_text'] = test_df.clean_title + ' ' + test_df.clean_question_body + ' ' + test_df.clean_answer
    all_texts = train_df['combine_text'].tolist() + test_df['combine_text'].tolist()
    all_texts = [text.split() for text in all_texts]
    model = Word2Vec(size = 300, min_count = 1, window = 5)
    model.build_vocab(all_texts)
    model.intersect_word2vec_format('../input/embedding-files/fasttext-wiki-news-subwords-300', lockf = 1)
    model.train(all_texts, total_examples = model.corpus_count, epochs = 5)
    counter = Counter()
    for text in all_texts:
      counter.update(text)
    vocab = {}
    vocab['token2id'] = {key:id+1 for id, (key, _) in enumerate(counter.items())}
    vocab['id2token'] = {value:key for key, value in vocab['token2id'].items()}
    vocab['word_freq'] = dict(counter)
    embedding_matrix = build_embedding_matrix(vocab, all_texts, 300, model)
else:
    file = np.load('embeddings.npz', allow_pickle = True)
    vocab = file['a'].tolist()
    embedding_matrix = file['b']

In [ ]:
MAX_LENGTH = 512
data['train_question_title'] = get_token_ids(train_df['clean_t_q'], MAX_LENGTH)
data['train_answer'] = get_token_ids(train_df['clean_answer'], MAX_LENGTH)
data['test_question_title'] = get_token_ids(test_df['clean_t_q'], MAX_LENGTH)
data['test_answer'] = get_token_ids(test_df['clean_answer'], MAX_LENGTH)

In [ ]:
train_fe = train_df[feature_engg_cols].values.astype(np.float32)
test_fe = test_df[feature_engg_cols].values.astype(np.float32)

In [ ]:
train_df['combine_title_question'] = train_df['question_title'] + ' ' + train_df['question_body']
test_df['combine_title_question'] = test_df['question_title'] + ' ' + test_df['question_body']

In [ ]:
encoder = hub.load('final_models/universal-sentence-encoder-qa_3')

data['train_question_title_use'] = []
data['train_answer_use'] = []

data['test_question_title_use'] = []
data['test_answer_use'] = []

BATCH_SIZE = 4

for i in range(0, train_df.shape[0], BATCH_SIZE):
  data['train_question_title_use'] += [encoder.signatures['question_encoder'](
      tf.constant(train_df['combine_title_question'].iloc[i:i+BATCH_SIZE].tolist())
      )['outputs'].numpy().astype(np.float16)]
  data['train_answer_use'] += [encoder.signatures['response_encoder'](
      input = tf.constant(train_df['answer'].iloc[i:i+BATCH_SIZE].tolist()), 
      context = tf.constant(train_df['answer'].iloc[i:i+BATCH_SIZE].tolist())
      )['outputs'].numpy().astype(np.float16)]
    
for i in range(0, test_df.shape[0], BATCH_SIZE):
  data['test_question_title_use'] += [encoder.signatures['question_encoder'](
      tf.constant(test_df['combine_title_question'].iloc[i:i+BATCH_SIZE].tolist())
      )['outputs'].numpy().astype(np.float16)]
  data['test_answer_use'] += [encoder.signatures['response_encoder'](
      input = tf.constant(test_df['answer'].iloc[i:i+BATCH_SIZE].tolist()), 
      context = tf.constant(test_df['answer'].iloc[i:i+BATCH_SIZE].tolist())
      )['outputs'].numpy().astype(np.float16)]
    

data['train_question_title_use'] = np.vstack(data['train_question_title_use'])
data['train_answer_use'] = np.vstack(data['train_answer_use'])

data['test_question_title_use'] = np.vstack(data['test_question_title_use'])
data['test_answer_use'] = np.vstack(data['test_answer_use'])

In [ ]:
def is_eng(x):
    if ((x == 'english') | (x == 'eli')):
        return 1
    return 0
def question_type(row):
    if ((row['is_eng'] == 1) & (row['a_count'] > 85)):
        return 1
    else:
        return 0

train_df['domain'] = train_df.host.apply(lambda s:s.split('.')[0])
test_df['domain'] = test_df.host.apply(lambda s:s.split('.')[0])
train_df['is_eng'] = train_df.domain.apply(lambda x:is_eng(x))
train_df['a_count'] = train_df.answer.apply(lambda x: len(x.split()))
train_df['question_type_spelling_modified'] = train_df[['is_eng', 'a_count']].apply(question_type, axis = 1)
test_df['is_eng'] = test_df.domain.apply(lambda x:is_eng(x))
test_df['a_count'] = test_df.answer.apply(lambda x: len(x.split()))
test_df['question_type_spelling_modified'] = test_df[['is_eng', 'a_count']].apply(question_type, axis = 1)

In [ ]:
def get_generator(X, batch_size = 32, training = True):
    Y = X[1]
    N = X[0][0].shape[0]
    if training == True:
        indexes = np.arange(N)
    else:
        indexes = np.arange(N)
    def generator():
        for i in indexes:
            yield {"input_1": X[0][0][i], "input_2": X[0][1][i],"input_3": X[0][2][i], "input_4": X[0][3][i], "input_5": X[0][4][i]}, Y[i]
    return tf.data.Dataset.from_generator(generator, 
    output_types = ({"input_1": tf.int32, "input_2": tf.int32,"input_3": tf.float16, "input_4": tf.float16, "input_5": tf.float32}, tf.float32)).repeat().batch(batch_size)

def SpearmanCorrCoeff(A, B):
  overall_score = 0
  for index in range(A.shape[1]):
      overall_score += spearmanr(A[:, index], B[:, index]).correlation
  return np.round(overall_score/A.shape[1], 3)

class PredictCallback(tf.keras.callbacks.Callback):
  def __init__(self, data, labels):
    self.data = data
    self.labels = labels
  def on_epoch_end(self, epoch, logs = {}):
    predictions = self.model.predict(self.data)
    print('\nValidation Score - ' + str(SpearmanCorrCoeff(self.labels, predictions)))

In [ ]:
def c_lstm(max_length):
  i1 = tf.keras.layers.Input(shape = (max_length))
  i2 = tf.keras.layers.Input(shape = (max_length))
  i3 = tf.keras.layers.Input(shape = (512), dtype = tf.float16)
  i4 = tf.keras.layers.Input(shape = (512), dtype = tf.float16)
  hand_features = tf.keras.layers.Input(shape = (train_fe.shape[1]), dtype = tf.float32)

  e1 = tf.keras.layers.Embedding(input_dim = embedding_matrix.shape[0], output_dim = embedding_matrix.shape[1], weights = [embedding_matrix], trainable = False)(i1)
  e2 = tf.keras.layers.Embedding(input_dim = embedding_matrix.shape[0], output_dim = embedding_matrix.shape[1], weights = [embedding_matrix], trainable = False)(i2)
  
  conv_1 = tf.keras.layers.Conv1D(filters = 300, kernel_size = 1, strides = 1)(e1)
  conv_2 = tf.keras.layers.Conv1D(filters = 300, kernel_size = 1, strides = 1)(e2)

  sd_i1 = tf.keras.layers.SpatialDropout1D(0.2)(conv_1)
  sd_i2 = tf.keras.layers.SpatialDropout1D(0.2)(conv_2)


  lstm_q_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True))(sd_i1)
  lstm_q_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True))(lstm_q_1)
    
  lstm_a_1 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True))(sd_i2)
  lstm_a_2 = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64, return_sequences = True))(lstm_a_1)

  max_pool_1 = tf.keras.layers.GlobalMaxPooling1D()(lstm_q_2)
  avg_pool_1 = tf.keras.layers.GlobalAveragePooling1D()(lstm_q_2)
  max_pool_2 = tf.keras.layers.GlobalMaxPooling1D()(lstm_a_2)
  avg_pool_2 = tf.keras.layers.GlobalAveragePooling1D()(lstm_a_2)

  
  concat = tf.keras.layers.Concatenate()([max_pool_1, max_pool_2, avg_pool_1, avg_pool_2, i3, i4, hand_features])
  
  dense = tf.keras.layers.Dense(784, activation = 'relu')(concat)
  
  drop = tf.keras.layers.Dropout(rate = 0.3)(dense)
  
  out = tf.keras.layers.Dense(30, activation = 'sigmoid')(drop)

  model = tf.keras.Model(inputs = [i1, i2, i3, i4, hand_features], outputs = [out])

  return model

In [ ]:
target_cols = ['question_asker_intent_understanding', 'question_body_critical', 'question_conversational', 'question_expect_short_answer', 
               'question_fact_seeking', 'question_has_commonly_accepted_answer', 'question_interestingness_others', 'question_interestingness_self', 
               'question_multi_intent', 'question_not_really_a_question', 'question_opinion_seeking', 'question_type_choice', 'question_type_compare', 
               'question_type_consequence', 'question_type_definition', 'question_type_entity', 'question_type_instructions', 'question_type_procedure', 
               'question_type_reason_explanation', 'question_type_spelling', 'question_well_written', 'answer_helpful', 'answer_level_of_information', 
               'answer_plausible', 'answer_relevance', 'answer_satisfaction', 'answer_type_instructions', 'answer_type_procedure', 'answer_type_reason_explanation', 
               'answer_well_written']
final_outputs = train_df[target_cols].values.astype(np.float32)

In [ ]:
if os.path.exists('final_models/c-lstm-save') == False:
    gkf = GroupKFold(n_splits = 5).split(X=train_df.url, groups = train_df.url)

    for fold, (train_idx, valid_idx) in enumerate(gkf):
        if fold != 2:
            continue
        prev_weights = None
        print(f"-------------------------------------------- Fold - {fold} ----------------------------------------------------")
        train_inputs = ((data['train_question_title'][train_idx], data['train_answer'][train_idx],
                        data['train_question_title_use'][train_idx], data['train_answer_use'][train_idx], train_fe[train_idx]), (final_outputs[train_idx]))
        valid_inputs = ((data['train_question_title'][valid_idx], data['train_answer'][valid_idx],
                        data['train_question_title_use'][valid_idx], data['train_answer_use'][valid_idx], train_fe[valid_idx]), (final_outputs[valid_idx]))
        BATCH_SIZE = 28
        train_dataset = get_generator(train_inputs)
        valid_dataset = tf.data.Dataset.from_tensor_slices(valid_inputs).batch(BATCH_SIZE)

        np.random.seed(SEED)
        rn.seed(SEED)
        tf.random.set_seed(SEED)
        model = c_lstm(MAX_LENGTH)
        optimizer = tf.keras.optimizers.Nadam(1*1e-3, beta_1=0.7, beta_2=0.777)
        model.compile(loss='binary_crossentropy', optimizer=optimizer)


        model.fit(train_dataset, epochs = 4, steps_per_epoch = train_idx.shape[0]//BATCH_SIZE, 
                    callbacks = [PredictCallback(valid_dataset, final_outputs[valid_idx])])
        
        break
else:
    c_lstm_model = tf.keras.models.load_model('final_models/c-lstm-save/c-lstm')

In [13]:
if os.path.exists('final_models/model-save-weights') == False:
    gkf = GroupKFold(n_splits = 5).split(X = train_df.url, groups = train_df.url)
    for fold, (train_idx, valid_idx) in enumerate(gkf):
        if fold != main_fold:
            continue
        final_outputs = train_df[cols].values.astype(np.float16)
        tf.keras.backend.clear_session()
        xlnet_train_inputs = (
                            data['xlnet_train_a'][tags[0]][train_idx], data['xlnet_train_t_q'][tags[0]][train_idx], data['xlnet_train_a'][tags[1]][train_idx], data['xlnet_train_t_q'][tags[1]][train_idx],
                        data['xlnet_train_a'][tags[2]][train_idx], data['xlnet_train_t_q'][tags[2]][train_idx]  
                        
                        )
        xlnet_valid_inputs = (
                            data['xlnet_train_a'][tags[0]][valid_idx], data['xlnet_train_t_q'][tags[0]][valid_idx], data['xlnet_train_a'][tags[1]][valid_idx], data['xlnet_train_t_q'][tags[1]][valid_idx],
                        data['xlnet_train_a'][tags[2]][valid_idx], data['xlnet_train_t_q'][tags[2]][valid_idx]  
                        
                        )
        
        xlnet_model = create_model()
        xlnet_model.compile(tf.keras.optimizers.Adam(learning_rate = 2*1e-5), loss = tf.keras.losses.BinaryCrossentropy())
        xlnet_model.fit(x = xlnet_train_inputs, y = final_outputs[train_idx], epochs = 3, batch_size = 4, steps_per_epoch = train_idx.shape[0]//4)
        tf.keras.backend.clear_session()
        print("\n####################################################################################################################\n")

        break
else:
    xlnet_model = transformer_model()
    xlnet_model.load_weights('final_models/model-save-weights/xlnet')

In [ ]:
class Optimize:
    def __init__(self):
        self.clips = [[0, 1] for i in range(30)]
        self.ab_ = [(0, 0.15), (0.85, 1)]
        self.new_scores, self.scores = (None, None)
    def fit(self, labels, preds):
        self.scores = [SpearmanCorrCoeff(labels[:, i:i+1], preds[:, i:i+1]) for i in range(30)]
        for i in range(30):
            self.golden_section_search(labels[:, i:i+1], preds[:, i:i+1], i, 0)
            self.golden_section_search(labels[:, i:i+1], preds[:, i:i+1], i, 1)
        self.new_scores = [np.nan_to_num(SpearmanCorrCoeff(labels[:, i:i+1], np.clip(preds[:, i:i+1], self.clips[i][0], self.clips[i][1]))) for i in range(30)]
        for i in range(30):
            if self.scores[i] >= self.new_scores[i]:
                self.clips[i] = [0, 1]
    def golden_section_search(self, labels, preds, i, idx):
        (a, b) = self.ab_[idx]
        c = 0.618
        x1 = b - c*(b-a)
        x2 = (b-a)*c + a
        
        for epochs in range(20):
            self.clips[i][idx] = x1
            score_a = -self.score(labels, preds, i)
            self.clips[i][idx] = x2
            score_b = -self.score(labels, preds, i)
            if np.isnan(score_a):
                score_a = 0
            elif np.isnan(score_b):
                score_b = 0
            elif score_a <= score_b:
                b = x2
                x2 = x1
                x1 = b - c*(b-a)
            else:
                a = x1
                x1 = x2
                x2 = (b-a)*c + a
        
        self.clips[i][idx] = x1
        score_x1 = self.score(labels, preds, i)
        self.clips[i][idx] = x2
        score_x2 = self.score(labels, preds, i)
        if score_x1 > score_x2:
            self.clips[i][idx] = x1
        else:
            self.clips[i][idx] = x2
                    
            
    def score(self, labels, preds, i):
        return SpearmanCorrCoeff(labels, np.clip(preds, self.clips[i][0], self.clips[i][1]))
    def transform(self, preds):
        temp = preds.copy()
        for i in range(30):
            clipped = np.clip(preds[:, i], self.clips[i][0], self.clips[i][1])
            if np.unique(clipped).shape[0] > 1:
                temp[:, i][:] = clipped
        return temp

In [ ]:
gkf = GroupKFold(n_splits = 5).split(X=train_df.url, groups = train_df.url)
for fold, (train_idx, valid_idx) in enumerate(gkf):
  if fold == 2:
    break

In [ ]:
train_y = final_outputs[train_idx]

valid_y = final_outputs[valid_idx]

c_lstm_inputs = (data['train_question_title'], data['train_answer'],
                   data['train_question_title_use'], data['train_answer_use'], train_fe)
c_lstm_predictions = c_lstm_model.predict(c_lstm_inputs)
c_lstm_train_predictions = c_lstm_predictions[train_idx]
c_lstm_valid_predictions = c_lstm_predictions[valid_idx]

xlnet_inputs = (
                    data['xlnet_train_a'][tags[0]], data['xlnet_train_t_q'][tags[0]], data['xlnet_train_a'][tags[1]], data['xlnet_train_t_q'][tags[1]],
                   data['xlnet_train_a'][tags[2]], data['xlnet_train_t_q'][tags[2]]  
                  
                 )
xlnet_predictions = xlnet_model.predict(xlnet_inputs)
xlnet_train_preds = xlnet_predictions[train_idx]
xlnet_valid_preds = xlnet_predictions[valid_idx]

In [38]:
print(f"Spearman Correlation Coefficient CNN-LSTM : {SpearmanCorrCoeff(c_lstm_valid_predictions, valid_y)}" )
print(f"Spearman Correlation Coefficient XLNET : {SpearmanCorrCoeff(xlnet_valid_preds, valid_y)}" )

Spearman Correlation Coefficient CNN-LSTM : 0.394
Spearman Correlation Coefficient XLNET : 0.398


# Applying Blending and Optimizer

In [ ]:
train_score_preds = 0.5*c_lstm_train_predictions + 0.5*xlnet_train_preds
valid_score_preds = 0.5*c_lstm_valid_predictions + 0.5*xlnet_valid_preds
train_score_preds[:, 19] = np.multiply(train_df['question_type_spelling_modified'].iloc[train_idx].values, train_score_preds[:, 19])
valid_score_preds[:, 19] = np.multiply(train_df['question_type_spelling_modified'].iloc[valid_idx].values, valid_score_preds[:, 19])

In [ ]:
opt = Optimize()
opt.fit(train_y, train_score_preds)
post_valid_predictions = opt.transform(valid_score_preds)
print(f"Validation Score Blending : {SpearmanCorrCoeff(valid_y, valid_score_preds)}")
print(f"Validation Score Blending + Optimizer {SpearmanCorrCoeff(valid_y, post_valid_predictions)}")

In [ ]:
from prettytable import PrettyTable
    
x = PrettyTable()

x.field_names = ["Modeling Strategy", "CV Score"]

x.add_row(["CNN-LSTM", 0.394])
x.add_row(["XLNET", 0.398])
x.add_row(["Blending", 0.424])
x.add_row(["Blending + Optimization", 0.442])

print(x)

# References
* https://www.kaggle.com/sakami/google-quest-single-lstm?scriptVersionId=28487242 - The CNN-LSTM takes some inspiration from this kernel
* https://www.kaggle.com/christofhenkel/how-to-preprocessing-when-using-embeddings - Preprocessing of text data is inspired from this kernel
* https://huggingface.co/ - Documentation for Huggingface Library
* https://www.kaggle.com/c/google-quest-challenge/discussion/130041 - Used the hack for question_type_spelling
